# 029 - Access - ILO 

**Created on:** June, 23 2018 by Andrew Lightner 

**First Analyst:** Andrew Lightner **|** 
**Date of Update:** 10 July 2018 **|**
**Date of Next Update:** N/A **|**
**Frequency:** Continuallly  

### **Outline: This notebook performs five tasks:** 
    
- 1) Document Changes to the download process. 
- 2) Access data using the Rilostat database implemented through rpy2 **A1 - July 2018**
- 3) Organize documentation of variables and selection of variables for the ESDB database. 


#### Import Packages

The code below imports the packages which will access the data. (imports a large number of macros) 

To run a particular cell, press ```ctrl+ENTER```

In [1]:
# python's calculation package 
from numpy import *
import numpy as np
# package for machine learning and economectrics 
import scipy as sp
# python's SAS/dataset manipulations package 
import pandas as pd

##########################
# package which allows python and R to interact almost seamlessly
##########################
# import R packages
from rpy2.robjects.packages import importr
# other R objects we may want to import 
import rpy2.robjects as ro
# allow pandas to read R datasets and activate this option.
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()


# selence warnings which are not of interest
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
pd.options.mode.chained_assignment = None

# 1. Access Data: Rilostat

This section accesses the ILOstat database using the Rilostat R package. This database will be transfered to a SAS database through the saspy package.

### Getting Started 

First, we load the R package into our python environment usign the rpy2 package. Afterwards, we will be able to use the R functions as if we were in an R environment. As a note, I merely openned an R environment to doadload the 'Rilostat' package on this computer. This is the easiest way to download packages and only needs to be done once. 

```install.packages("Rilostat")```

Second, we open the 'Getting Started' documentation which outlines clearly how to use the Rilostat package. 

#### Import R Packages 

In [2]:
stats = importr('stats')
ILO = importr('Rilostat')

In [3]:
# this code brings an html to your browser with sample code and documentation
ILO.ilostat('GettingStarted')

rpy2.rinterface.NULL

### Search for data 

To access the table of contents of all available indicators in ILOSTAT (by indicator). Now, we can search for indicators by their IDs or their labels, and then use their associated indicator values to get the datasets.  Also, Rilostat indicates whether the values are collected or estimated along with the year ranges for the indicators of interest.  Finally, it includes when the data was last updated in the ``` last.update ``` column. 

In [3]:
# access data documentation from ILO 
data_options = pandas2ri.ri2py(ILO.get_ilostat_toc())

In [6]:
# save file to Mappings folder for future use. 
file = 'C:/Users/alightner/Documents/Source Updates/029 ILO/Mappings/'

# send options to a csv
data_options.to_csv(file+'ILO_allindicators.csv')

In [7]:
# top five observation in dataset
data_options.head()

,id,indicator,indicator.label,freq,freq.label,size,data.start,data.end,last.update,n.records,collection,collection.label,subject,subject.label
1,GDP_211P_NOC_NB_A,GDP_211P_NOC_NB,Output per worker (GDP constant 2011 internati...,A,Annual,75.06 KB,1991,2022,06/06/2018 10:31:08,8736.0,ILOEST,ILO estimates,LPY,Labour productivity
2,GDP_205U_NOC_NB_A,GDP_205U_NOC_NB,Output per worker (GDP constant 2010 US $) -- ...,A,Annual,73.62 KB,1991,2022,06/06/2018 10:31:08,8736.0,ILOEST,ILO estimates,LPY,Labour productivity
3,POP_2LDR_NOC_RT_A,POP_2LDR_NOC_RT,Labour dependency ratio -- ILO modelled estima...,A,Annual,80.85 KB,1991,2022,06/06/2018 10:27:40,8736.0,ILOEST,ILO estimates,ILOEST,ILO modelled estimates
4,POP_2POP_SEX_AGE_NB_A,POP_2POP_SEX_AGE_NB,Population by sex and age -- UN estimates and ...,A,Annual,8.76 MB,1990,2030,25/01/2018 09:32:38,1108107.0,ILOEST,ILO estimates,ILOEST,ILO modelled estimates
5,POP_2POP_SEX_AGE_GEO_NB_A,POP_2POP_SEX_AGE_GEO_NB,"Population by sex, age and rural / urban areas...",A,Annual,3.99 MB,1990,2015,31/05/2018 14:45:08,486486.0,ILOEST,ILO estimates,ILOEST,ILO modelled estimates


Here we list all the indicator labels to see all the data available through this API package. There are quite a few...

In [8]:
def search_ind(data, text = '', return_type = 'dataframe'):
    '''return options are dataframe for all data or list for list of indicator labels'''
    
    # select all observations where the indicator label contains the text
    data = data[data['indicator.label'].str.contains(text)] 
    
    # turn the results into a list of indicator labels if return_type =='list'
    if return_type=='list': 
        # turn pandas series into a list 
        data = list(data['indicator.label'])
    
    # return the data 
    return data

In [9]:
search_ind(data_options, 'Population')

,id,indicator,indicator.label,freq,freq.label,size,data.start,data.end,last.update,n.records,collection,collection.label,subject,subject.label
4,POP_2POP_SEX_AGE_NB_A,POP_2POP_SEX_AGE_NB,Population by sex and age -- UN estimates and ...,A,Annual,8.76 MB,1990,2030,25/01/2018 09:32:38,1108107.0,ILOEST,ILO estimates,ILOEST,ILO modelled estimates
5,POP_2POP_SEX_AGE_GEO_NB_A,POP_2POP_SEX_AGE_GEO_NB,"Population by sex, age and rural / urban areas...",A,Annual,3.99 MB,1990,2015,31/05/2018 14:45:08,486486.0,ILOEST,ILO estimates,ILOEST,ILO modelled estimates
6,POP_2POP_GEO_NB_A,POP_2POP_GEO_NB,Population by rural / urban areas -- UN estima...,A,Annual,300.08 KB,1990,2030,31/05/2018 14:45:05,33579.0,ILOEST,ILO estimates,ILOEST,ILO modelled estimates


## Get SDG Data 

In the code below, we generate a short function which selects indicators based on the first three letters of the id code. We then access all SGD variables by selecting all veraibles which begin with the three letters 'SDG'. 

###### Define function

In [10]:
def search_id3(data, text = '', return_type = 'dataframe'): 
    ''' This function search indicators based on their first three characters 
    options include: dataframe, list_lab, list_ids. list lab returns the labels while list_ids returns the ids'''
    
    # select the first 3 characters in each observations of id 
    data['id3'] = data['id'].str[0:3]
    
    # select only those observations where the first 3 is equal to the provided text
    data= data[data['id3'] == text]
    
    # return just the id values if specified
    if return_type =='list_ids': 
        # select just the values of the column data['id']
        data = data['id'].values
        
    # return just the indicator labels if specified
    if return_type == 'list_lab': 
        # select only the values of the indicator label column in data 
        data = data['indicator.label'].values
    
    # return the data 
    return data

###### Select Data

Here we use the function above to find all SDG varaibles, we return the function as a list. 

In [11]:
# search indicators 
SDG_ids = search_id3(data_options, 'SDG', return_type = 'list_ids') 
# show results 
SDG_ids

array(['SDG_0111_SEX_AGE_RT_A', 'SDG_0131_SEX_SOC_RT_A',
       'SDG_0552_OCU_RT_A', 'SDG_0821_NOC_RT_A', 'SDG_A831_SEX_RT_A',
       'SDG_B831_SEX_RT_A', 'SDG_0851_SEX_OCU_NB_A',
       'SDG_0852_SEX_AGE_RT_A', 'SDG_0852_SEX_DSB_RT_A',
       'SDG_0861_SEX_RT_A', 'SDG_0871_SEX_AGE_NB_A',
       'SDG_0871_SEX_AGE_RT_A', 'SDG_N881_SEX_MIG_RT_A',
       'SDG_F881_SEX_MIG_RT_A', 'SDG_0922_NOC_RT_A', 'SDG_1041_NOC_RT_A'],
      dtype=object)

## ILO Estimates 

Here we select all ILO estimate variables. Although these indicators are widely known to have methodological flaws, they also provide a large time-series and may have some usefulness moving forward for rough estimates of economic acitivity in a country. The quality of other indicators will likely be very country-year specific.  Thus, non-estimate indicators can be accessed using this package should the economist need to access the data. 

In [17]:
# return all indicator labels for all annual level ILO estimates 
estimates = data_options[(data_options['subject.label']=='ILO modelled estimates') & # ILO estimates 
             (~data_options['indicator.label'].str.contains('Population'))& # not population (signaled by the ~)
             (data_options['freq.label']=='Annual')]['indicator'].values # annual data type
estimates

array(['POP_2LDR_NOC_RT', 'EAP_2EAP_SEX_AGE_NB', 'EAP_2MDN_SEX_NB',
       'EAP_2WAP_SEX_AGE_RT', 'EMP_2EMP_SEX_AGE_NB',
       'EMP_2EMP_SEX_STE_NB', 'EMP_2EMP_SEX_ECO_NB',
       'EMP_2EMP_SEX_OCU_NB', 'EMP_2EMP_SEX_AGE_CLA_NB',
       'EMP_2EMP_SEX_STE_DT', 'EMP_2EMP_SEX_ECO_DT',
       'EMP_2EMP_SEX_OCU_DT', 'EMP_2EMP_SEX_AGE_CLA_DT',
       'EMP_2WAP_SEX_AGE_RT', 'UNE_2UNE_SEX_AGE_NB',
       'UNE_2EAP_SEX_AGE_RT', 'EIP_2EIP_SEX_AGE_NB',
       'EIP_2WAP_SEX_AGE_RT', 'EAR_MREE_NOC_GR'], dtype=object)

Place all ID's from the list above into a list, loop over the list and append to a dataframe. 

In [18]:
estimated_ids = list(estimates)

## ESDB Indicators

There are a set of indicators which existed in the previous set of ESDB indicators. We will import this list and ensure that the data we selected includes these indicators. 

As shown by the list of estimated indicators above, we already have accessesd the employment and labour force participation by sex and economic activity. Thus, we already have the variables of interest within our dataset. 

However, we must ensure that these variables recieve the correct series_id as the names may have shifted due to a different download pattern. 

In [15]:
file = 'C:/Users/alightner/Documents/Source Updates/029 ILO/Documentation/'

series_inapps = pd.read_excel(file+'In_apps.xlsx')
series_inapps

,source_id,series_id,series_name,product
0,29,1493,"Labor force participation rate, total",IGD Filter (GF)
1,29,1494,"Labor force participation rate, women",IGD Filter (GF)
2,29,1496,"Labor force participation rate, men",IGD Filter (GF)
3,29,1499,"Employment, total (thousands)",IGD Filter (GF)
4,29,1500,"Employment, women (thousands)",IGD Filter (GF)
5,29,1501,"Employment, men (thousands)",IGD Filter (GF)
6,29,1503,"Employees by industry, agriculture (thousands)",IGD Filter (GF)
7,29,1507,"Employees by industry, construction (thousands)",IGD Filter (GF)
8,29,1508,"Employees by industry, manufacturing (thousands)",IGD Filter (GF)
9,29,1511,"Employees by industry, mining and quarrying; e...",IGD Filter (GF)


## Get Data 

First we combine the list of of ID names of the SDG list and the estimated IDS. Then, we generate a dataset using the first observations in the list ```full_ids```. Through a loop, we append each other indicator to this initial dataset. 

In [4]:
all_ids = ['SDG_0111_SEX_AGE_RT_A', 'SDG_0131_SEX_SOC_RT_A', 'SDG_0552_OCU_RT_A', 'SDG_0821_NOC_RT_A', 'SDG_A831_SEX_RT_A', 'SDG_B831_SEX_RT_A', 'SDG_0851_SEX_OCU_NB_A', 'SDG_0852_SEX_AGE_RT_A', 'SDG_0852_SEX_DSB_RT_A', 'SDG_0861_SEX_RT_A', 'SDG_0871_SEX_AGE_NB_A', 'SDG_0871_SEX_AGE_RT_A', 'SDG_N881_SEX_MIG_RT_A', 'SDG_F881_SEX_MIG_RT_A', 'SDG_0922_NOC_RT_A', 'SDG_1041_NOC_RT_A', 'EAP_2EAP_SEX_AGE_NB_A', 'EAP_2MDN_SEX_NB_A', 'EAP_2WAP_SEX_AGE_RT_A', 'EMP_2EMP_SEX_AGE_NB_A', 'EMP_2EMP_SEX_STE_NB_A', 'EMP_2EMP_SEX_ECO_NB_A', 'EMP_2EMP_SEX_OCU_NB_A', 'EMP_2EMP_SEX_AGE_CLA_NB_A', 'EMP_2EMP_SEX_STE_DT_A', 'EMP_2EMP_SEX_ECO_DT_A', 'EMP_2EMP_SEX_OCU_DT_A', 'EMP_2EMP_SEX_AGE_CLA_DT_A', 'EMP_2WAP_SEX_AGE_RT_A', 'UNE_2UNE_SEX_AGE_NB_A', 'UNE_2EAP_SEX_AGE_RT_A', 'EIP_2EIP_SEX_AGE_NB_A', 'EIP_2WAP_SEX_AGE_RT_A', 'EAR_MREE_NOC_GR_A']

### Download Data

In [10]:

# create a dataframe with the first observation in the list above, others will be appended to this dataset. 
df_full = pandas2ri.ri2py(ILO.get_ilostat(id=all_ids[0], time_format='num', detail='dataonly', type='both'))

# loop over and append to dataset
for i in all_ids[1:]: 
    
    df = pandas2ri.ri2py(ILO.get_ilostat(id=i, time_format='num', detail='dataonly', type='both'))
    old_len = len(df_full)
    df_full = df_full.append(df, sort=False)
    print('Lenth of new dataset: '+str(len(df_full))+'. Added ' + i)

Lenth of new dataset: 31847. Added SDG_0131_SEX_SOC_RT_A
Lenth of new dataset: 33767. Added SDG_0552_OCU_RT_A
Lenth of new dataset: 38101. Added SDG_0821_NOC_RT_A
Lenth of new dataset: 38565. Added SDG_A831_SEX_RT_A
Lenth of new dataset: 39348. Added SDG_B831_SEX_RT_A
Lenth of new dataset: 44977. Added SDG_0851_SEX_OCU_NB_A
Lenth of new dataset: 65739. Added SDG_0852_SEX_AGE_RT_A
Lenth of new dataset: 66733. Added SDG_0852_SEX_DSB_RT_A
Lenth of new dataset: 69871. Added SDG_0861_SEX_RT_A
Lenth of new dataset: 70948. Added SDG_0871_SEX_AGE_NB_A
Lenth of new dataset: 72400. Added SDG_0871_SEX_AGE_RT_A
Lenth of new dataset: 72711. Added SDG_N881_SEX_MIG_RT_A
Lenth of new dataset: 75613. Added SDG_F881_SEX_MIG_RT_A
Lenth of new dataset: 77343. Added SDG_0922_NOC_RT_A
Lenth of new dataset: 78099. Added SDG_1041_NOC_RT_A
Lenth of new dataset: 1018311. Added EAP_2EAP_SEX_AGE_NB_A
Lenth of new dataset: 1042296. Added EAP_2MDN_SEX_NB_A
Lenth of new dataset: 1982508. Added EAP_2WAP_SEX_AGE_RT_A


#### Drop weird observatsion without year 

In [11]:
df_full[~df_full['time'].notnull()]

,collection,collection.label,ref_area,ref_area.label,source,source.label,indicator,indicator.label,sex,sex.label,classif1,classif1.label,time,obs_value,classif2,classif2.label


In [12]:
# delete odd observation where df_full['time'] is missing 
df_full = df_full[df_full['time'].notnull()]

In [13]:
len(df_full)

5887045

Change the data type of the 'year' variable time to integer. 

In [14]:
# change year (time) type to int 
df_full['time'] = df_full['time'].astype('int')
df_full.head()

,collection,collection.label,ref_area,ref_area.label,source,source.label,indicator,indicator.label,sex,sex.label,classif1,classif1.label,time,obs_value,classif2,classif2.label
1,SDG,SDG labour market indicators,AFG,Afghanistan,XA:2198,AFG - ILO - ILO Estimates and Projections,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_YGE15,Age: 15+,2000,79.624,NaN,NaN
2,SDG,SDG labour market indicators,AFG,Afghanistan,XA:2198,AFG - ILO - ILO Estimates and Projections,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_Y15-24,Age: 15-24,2000,78.465,NaN,NaN
3,SDG,SDG labour market indicators,AFG,Afghanistan,XA:2198,AFG - ILO - ILO Estimates and Projections,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_YGE25,Age: 25+,2000,80.118,NaN,NaN
4,SDG,SDG labour market indicators,AFG,Afghanistan,XA:2198,AFG - ILO - ILO Estimates and Projections,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_M,Sex: Male,AGE_YTHADULT_YGE15,Age: 15+,2000,79.117,NaN,NaN
5,SDG,SDG labour market indicators,AFG,Afghanistan,XA:2198,AFG - ILO - ILO Estimates and Projections,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_M,Sex: Male,AGE_YTHADULT_Y15-24,Age: 15-24,2000,77.887,NaN,NaN


## Save raw data as CSV in raw datafile 

In [16]:
file = 'C:/Users/alightner/Documents/Source Updates/029 ILO/Raw data/'

df_full.to_csv(file+'029_rawdata.csv')

# save a small version as a sample (first two hundred)
df_full.iloc[0:200, :].to_csv(file+'029_rawdata_small.csv')

# 2. Save documentation

Here we generate an excel directory for all variables in our final dataset, as well as a directory for just those which will be added to the ESDB. 

In [3]:
# upload df_full from csv 
file = 'C:/Users/alightner/Documents/Source Updates/029 ILO/Raw data/'
# specify low memory = False due to the save of the string columns 
df_full = pd.read_csv(file+'029_rawdata.csv', low_memory=False)

### 2.1.1 Create Directory

There are several options for age categories which are not necessary for our analysis. We keep only the 10 year age categories. 

In [4]:
# select only the variables we are interested in . 
directory = df_full[['collection', 'collection.label', 'indicator', 'indicator.label', 'sex', 'sex.label', 'classif1', 'classif1.label', 'classif2', 'classif2.label']]

# drop all duplicates to generate a source file
directory = directory.drop_duplicates()

# This shows we have around 893 unique indicators (mostly due to sex and age dissagregation)
len(directory)

893

In [5]:
directory.head()

,collection,collection.label,indicator,indicator.label,sex,sex.label,classif1,classif1.label,classif2,classif2.label
0,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_YGE15,Age: 15+,NaN,NaN
1,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_Y15-24,Age: 15-24,NaN,NaN
2,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_YGE25,Age: 25+,NaN,NaN
3,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_M,Sex: Male,AGE_YTHADULT_YGE15,Age: 15+,NaN,NaN
4,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_M,Sex: Male,AGE_YTHADULT_Y15-24,Age: 15-24,NaN,NaN


There is some interesting data in the data_options dataset such as the beginning and end years of the time series and the date for the last update. We merge in this data. 

In [12]:
directory_final = pd.merge(directory, data_options[['indicator', 'freq.label', 'data.start', 'data.end', 'last.update', 'subject.label']], on='indicator', how='inner')

directory_final.head()

,collection,collection.label,indicator,indicator.label,sex,sex.label,classif1,classif1.label,classif2,classif2.label,freq.label,data.start,data.end,last.update,subject.label
0,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_YGE15,Age: 15+,NaN,NaN,Annual,2000,2022,22/07/2018 07:59:07,SDG labour market indicators
1,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_Y15-24,Age: 15-24,NaN,NaN,Annual,2000,2022,22/07/2018 07:59:07,SDG labour market indicators
2,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_T,Sex: Total,AGE_YTHADULT_YGE25,Age: 25+,NaN,NaN,Annual,2000,2022,22/07/2018 07:59:07,SDG labour market indicators
3,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_M,Sex: Male,AGE_YTHADULT_YGE15,Age: 15+,NaN,NaN,Annual,2000,2022,22/07/2018 07:59:07,SDG labour market indicators
4,SDG,SDG labour market indicators,SDG_0111_SEX_AGE_RT,[1.1.1] Working poverty rate (percentage of em...,SEX_M,Sex: Male,AGE_YTHADULT_Y15-24,Age: 15-24,NaN,NaN,Annual,2000,2022,22/07/2018 07:59:07,SDG labour market indicators


### Save Data

In [ ]:
file = 'C:/Users/alightner/Documents/Source Updates/029 ILO/Mappings/'

directory_final.to_csv(file+'029_series_documentation.csv')